# GenoMix Mamba 2

In [1]:
import sys

from transformers import Mamba2Config
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append('..')

from genomix.models.genomix_vallim2 import GenoMixMamba2Model, GenoMixMamba2ForCausalLM
from genomix.models.genomix_config import GenoMixMamba2Config

In [2]:
config = GenoMixMamba2Config()
d_model = config.d_model
ssm_headdim = config.ssm_cfg['headdim']
assert d_model / ssm_headdim % 8 == 0, f'd_model / ssm_headdim % 8 must be 0'

attn_num_heads = config.attn_cfg['num_heads']
config.attn_cfg['head_dim'] = d_model // attn_num_heads
config.attn_cfg['rotary_emb_dim'] = d_model // attn_num_heads

print(config)

GenoMixMamba2Config {
  "attn_cfg": {
    "causal": true,
    "d_conv": 4,
    "head_dim": 32,
    "num_heads": 8,
    "out_proj_bias": false,
    "qkv_proj_bias": false,
    "rotary_emb_dim": 32
  },
  "attn_layer_idx": [],
  "bos_token_id": 0,
  "d_intermediate": 0,
  "d_model": 256,
  "eos_token_id": 2,
  "fused_add_norm": true,
  "initializer_cfg": {
    "initializer_range": 0.1,
    "rescale_prenorm_residual": false
  },
  "input_embedding_cfg": {
    "fusion_type": "add",
    "use_tabular_embedding": false
  },
  "model_type": "genomix-mamba2",
  "moe_cfg": null,
  "moe_layer_idx": [],
  "n_layers": 32,
  "norm_epsilon": 1e-05,
  "pad_token_id": 2,
  "pad_vocab_size_multiple": 8,
  "residual_in_fp32": true,
  "rms_norm": true,
  "ssm_cfg": {
    "A_init_range": [
      1,
      16
    ],
    "D_has_hdim": false,
    "bias": false,
    "chunk_size": 256,
    "conv_bias": true,
    "conv_init": null,
    "d_conv": 4,
    "d_ssm": null,
    "d_state": 64,
    "dt_init_floor": 0.0001

In [3]:
model = GenoMixMamba2ForCausalLM(config).to('cuda')

model

UnboundLocalError: local variable 'use_tabular_embedding' referenced before assignment

In [4]:
model

GenomixMamba2(
  (model): Mamba2ForCausalLM(
    (backbone): Mamba2Model(
      (embeddings): Embedding(16, 512)
      (layers): ModuleList(
        (0-63): 64 x Mamba2Block(
          (norm): Mamba2RMSNorm()
          (mixer): Mamba2Mixer(
            (act): SiLU()
            (conv1d): Conv1d(3072, 3072, kernel_size=(4,), stride=(1,), padding=(3,), groups=3072)
            (in_proj): Linear(in_features=512, out_features=4104, bias=False)
            (norm): MambaRMSNormGated()
            (out_proj): Linear(in_features=1024, out_features=512, bias=False)
          )
        )
      )
      (norm_f): Mamba2RMSNorm()
    )
    (lm_head): Linear(in_features=512, out_features=16, bias=False)
  )
)

In [5]:
input_id1 = id(model.model.backbone.embeddings.weight)
input_id2 = id(model.model.lm_head.weight)

print(input_id1==input_id2)

True


**NOTE**: if `safe_serialization=True` (default setting) in `model.save_pretrained`,
There will raise error:

```python
RuntimeError: The weights trying to be saved contained shared tensors [{'model.lm_head.weight', 'model.backbone.embeddings.weight'}] that are mismatching the transformers base configuration. Try saving using `safe_serialization=False` or remove this tensor sharing.
```

In [6]:

print(GenomixMamba2.config_class)
# model.save_pretrained('/home/share/huadjyin/home/baiyong01/projects/genomix/tmp/models', safe_serialization=False)

<class 'transformers.models.mamba2.configuration_mamba2.Mamba2Config'>


In [7]:
config = Mamba2Config.from_pretrained('/home/share/huadjyin/home/baiyong01/projects/genomix/tmp/models/config.json')

In [8]:
config

Mamba2Config {
  "_name_or_path": "genomix_mamba2",
  "architectures": [
    "GenomixMamba2"
  ],
  "bos_token_id": 0,
  "chunk_size": 256,
  "conv_kernel": 4,
  "d_model": 512,
  "eos_token_id": 2,
  "expand": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 512,
  "initializer_range": 0.1,
  "intermediate_size": 512,
  "layer_norm_epsilon": 1e-05,
  "model_type": "mamba2",
  "n_groups": 8,
  "n_head": 8,
  "n_layer": 24,
  "num_heads": 8,
  "num_hidden_layers": 64,
  "num_layers": 24,
  "pad_token_id": 1,
  "rescale_prenorm_residual": false,
  "residual_in_fp32": true,
  "rms_norm": true,
  "state_size": 128,
  "tie_word_embeddings": true,
  "time_step_floor": 0.0001,
  "time_step_limit": [
    0.0,
    Infinity
  ],
  "time_step_max": 0.1,
  "time_step_min": 0.001,
  "time_step_rank": 256,
  "torch_dtype": "float32",
  "transformers_version": "4.46.3",
  "use_bias": false,
  "use_cache": true,
  "use_conv_bias": true,
  "vocab_size": 16
}

In [9]:
model = GenomixMamba2.from_pretrained(
    '/home/share/huadjyin/home/baiyong01/projects/genomix/tmp/models',local_files_only=True)

In [9]:
model

GenomixMamba2(
  (model): Mamba2ForCausalLM(
    (backbone): Mamba2Model(
      (embeddings): Embedding(16, 512)
      (layers): ModuleList(
        (0-63): 64 x Mamba2Block(
          (norm): Mamba2RMSNorm()
          (mixer): Mamba2Mixer(
            (act): SiLU()
            (conv1d): Conv1d(3072, 3072, kernel_size=(4,), stride=(1,), padding=(3,), groups=3072)
            (in_proj): Linear(in_features=512, out_features=4104, bias=False)
            (norm): MambaRMSNormGated()
            (out_proj): Linear(in_features=1024, out_features=512, bias=False)
          )
        )
      )
      (norm_f): Mamba2RMSNorm()
    )
    (lm_head): Linear(in_features=512, out_features=16, bias=False)
  )
)

In [13]:
model.model.backbone.embeddings.weight.shape

torch.Size([16, 512])

In [17]:
model.base_model_prefix

'model.backbone'

In [21]:
from transformers import JetMoeConfig

jet_config = JetMoeConfig()
jet_config

JetMoeConfig {
  "activation_function": "silu",
  "attention_dropout": 0.0,
  "aux_loss_coef": 0.01,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_size": 2048,
  "initializer_range": 0.01,
  "intermediate_size": 5632,
  "kv_channels": 128,
  "max_position_embeddings": 4096,
  "model_type": "jetmoe",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidden_layers": 12,
  "num_key_value_heads": 16,
  "num_local_experts": 8,
  "output_router_logits": false,
  "rms_norm_eps": 1e-06,
  "rope_theta": 10000.0,
  "transformers_version": "4.46.3",
  "use_cache": true,
  "vocab_size": 32000
}